### LSTM Autoencoder

Previous example: [/examples/gan/intro.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/gan/intro.ipynb)  
Modified from: [A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)  
Next example: [/examples/time_series/autoencoder.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/autoencoder.ipynb)

In [1]:
from numpy import linspace
from keras.models import Sequential, Model
from keras.layers import LSTM, RepeatVector, Dense, TimeDistributed

In [2]:
dim_of_in = 32
pred_shift = 2
extra_out_dim = 3
extra_end = pred_shift + extra_out_dim
seq_all = linspace(-1, 1, dim_of_in + extra_end)
seq_all = seq_all.reshape((1, -1, 1)) # [samples, timesteps, features]
seq_in = seq_all[:, :-extra_end, :]
seq_out = seq_all[:, pred_shift:, :]
dim_of_out = len(seq_out[0, :, 0])
def disp(v1, v2, n1, n2):
    i0 = extra_end + 2
    print('First elements:')
    for i1 in range(i0): print(f'{i1:<20}{n1}: {v1[0, i1, 0]:.3f}, {n2}: {v2[0, i1, 0]:.3f}')
    print('\nLast elements:')
    for i1 in reversed(range(i0)): i2 = i1 + 1; print(f'{-i2:<20}{n1}: {v1[0, -i2, 0]:.3f}, {n2}: {v2[0, -i2, 0]:.3f}')

disp(seq_in, seq_out, 'in', 'out')
del pred_shift, extra_out_dim, seq_all

First elements:
0                   in: -1.000, out: -0.889
1                   in: -0.944, out: -0.833
2                   in: -0.889, out: -0.778
3                   in: -0.833, out: -0.722
4                   in: -0.778, out: -0.667
5                   in: -0.722, out: -0.611
6                   in: -0.667, out: -0.556

Last elements:
-7                  in: 0.389, out: 0.667
-6                  in: 0.444, out: 0.722
-5                  in: 0.500, out: 0.778
-4                  in: 0.556, out: 0.833
-3                  in: 0.611, out: 0.889
-2                  in: 0.667, out: 0.944
-1                  in: 0.722, out: 1.000


In [3]:
model = Sequential()
model.add(LSTM(16, input_shape=(dim_of_in, 1), name='encoder'))
model.add(RepeatVector(dim_of_out))
model.add(LSTM(16, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-01-04 10:25:17.040218: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-04 10:25:17.040336: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (LSTM)              (None, 16)                1152      
                                                                 
 repeat_vector (RepeatVector  (None, 35, 16)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 35, 16)            2112      
                                                                 
 time_distributed (TimeDistr  (None, 35, 1)            17        
 ibuted)                                                         
                                                                 
Total params: 3,281
Trainable params: 3,281
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.fit(seq_in, seq_out, epochs=128, verbose=0)

2023-01-04 10:25:17.481353: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-04 10:25:18.510999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:18.673101: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:18.765173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:18.879252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:19.062073: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [5]:
seq_pred = model.predict(seq_in, verbose=0)
disp(seq_out, seq_pred, 'out', 'pred')

First elements:
0                   out: -0.889, pred: -0.512
1                   out: -0.833, pred: -0.741
2                   out: -0.778, pred: -0.816
3                   out: -0.722, pred: -0.804
4                   out: -0.667, pred: -0.747
5                   out: -0.611, pred: -0.671
6                   out: -0.556, pred: -0.589

Last elements:
-7                  out: 0.667, pred: 0.687
-6                  out: 0.722, pred: 0.749
-5                  out: 0.778, pred: 0.811
-4                  out: 0.833, pred: 0.872
-3                  out: 0.889, pred: 0.931
-2                  out: 0.944, pred: 0.985
-1                  out: 1.000, pred: 1.036


2023-01-04 10:25:21.973593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:22.029186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:22.085078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [6]:
encoder = Model(inputs=model.inputs, outputs=model.get_layer('encoder').output)
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 1)]           0         
                                                                 
 encoder (LSTM)              (None, 16)                1152      
                                                                 
Total params: 1,152
Trainable params: 1,152
Non-trainable params: 0
_________________________________________________________________


In [7]:
seq_encoded = encoder.predict(seq_in, verbose=0)
print(f'{seq_in.shape = }, {seq_encoded.shape = }')

seq_in.shape = (1, 32, 1), seq_encoded.shape = (1, 16)


2023-01-04 10:25:22.309579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-04 10:25:22.339579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Previous example: [/examples/gan/intro.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/gan/intro.ipynb)  
Modified from: [A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)  
Next example: [/examples/time_series/autoencoder.ipynb](https://github.com/serhatsoyer/py4ML/blob/main/examples/time_series/autoencoder.ipynb)